In [ ]:
# CELL 13b-DEBUG-MINIMAL - Verify degenerate-label handling (30s smoke test)
import numpy as np
import cupy as cp
from unittest.mock import MagicMock

# Test function (final bulletproof version)
def safe_predict_proba_gpu(clf, x_gpu):
    W, b = None, None
    n_features = x_gpu.shape[1]
    
    if hasattr(clf, 'multiclass_estimator'):
        m_est = clf.multiclass_estimator
        if hasattr(m_est, 'coef_') and m_est.coef_ is not None:
            W = cp.asarray(m_est.coef_, dtype=cp.float32)
            b = cp.asarray(m_est.intercept_, dtype=cp.float32)
        elif hasattr(m_est, 'estimators_'):
            ws, bs = [], []
            for e in m_est.estimators_:
                # Check if this estimator is degenerate
                if not hasattr(e, 'coef_') or e.coef_ is None:
                    # DEGENERATE: zero weights
                    w = cp.zeros((1, n_features), dtype=cp.float32)
                    b_val = -10.0
                else:
                    # VALID: extract weights
                    w = cp.asarray(e.coef_, dtype=cp.float32)
                    if w.ndim == 1:
                        w = w.reshape(1, -1)
                    b_val = float(cp.asarray(e.intercept_, dtype=cp.float32).ravel()[0])
                
                ws.append(w)
                bs.append(b_val)
            
            W = cp.vstack(ws)  # (n_classes, n_features)
            b = cp.array(bs, dtype=cp.float32)  # (n_classes,)

    if W is None and hasattr(clf, 'coef_') and clf.coef_ is not None:
        W = cp.asarray(clf.coef_, dtype=cp.float32)
        b = cp.asarray(clf.intercept_, dtype=cp.float32)
    
    if W is not None and b is not None:
        if W.ndim == 1:
            W = W.reshape(1, -1)
        scores = cp.dot(x_gpu, W.T) + b
        return 1.0 / (1.0 + cp.exp(-cp.clip(scores, -50.0, 50.0)))
    
    n_samples = x_gpu.shape[0]
    return cp.zeros((n_samples, 1), dtype=cp.float32)

# --- TESTS ---
print('Test 1: Degenerate estimator (no coef_) → fallback to ~0')
X_gpu = cp.random.randn(5, 10, dtype=cp.float32)
clf = MagicMock()
clf.multiclass_estimator = MagicMock()
clf.multiclass_estimator.estimators_ = []

for i in range(3):
    est = MagicMock()
    if i == 1:
        est.coef_ = None  # Degenerate
    else:
        est.coef_ = cp.random.randn(1, 10).astype(cp.float32)
        est.intercept_ = cp.array([0.0], dtype=cp.float32)
    clf.multiclass_estimator.estimators_.append(est)

result = safe_predict_proba_gpu(clf, X_gpu)
assert result.shape == (5, 3), f'Expected (5,3), got {result.shape}'
assert cp.all(result[:, 1] < 0.01), f'Degenerate class prob={float(result[:, 1].max()):.3f} (expected ~0)'
print('✓ PASS: Degenerate class has ~0 probability')

print('\nTest 2: Valid weights → finite probabilities')
clf2 = MagicMock()
clf2.multiclass_estimator = MagicMock()
clf2.multiclass_estimator.coef_ = cp.random.randn(3, 10).astype(cp.float32)
clf2.multiclass_estimator.intercept_ = cp.random.randn(3).astype(cp.float32)

result2 = safe_predict_proba_gpu(clf2, X_gpu)
assert cp.isfinite(result2).all(), 'Non-finite values detected'
assert cp.all((result2 >= 0) & (result2 <= 1)), 'Probs outside [0,1]'
print('✓ PASS: Valid weights produce finite [0,1] probs')

print('\n✓ All checks passed. Fix verified.')

ModuleNotFoundError: No module named 'cupy'